In [1]:
import MySQLdb

import import_ipynb

from user import User
from decimal import Decimal

def student_menu3(student, db):
    #TODO modify below(함수에 아무것도 없으면 오류가 생겨서 아무 문장이나 적어놨습니다.)
    try:
        c = db.cursor()
        #print("11")
        c.execute('SELECT student.dept_name, credits, grade from student left outer join takes on (student.ID = takes.ID) '
                  + 'left outer join course on (takes.course_id = course.course_id) where'
              + ' student.ID = "' + student.id + '";')
        data = c.fetchall()
        #print(data)
        #print("22")
    
        dept= data[0][0]
        creds = 0
        sumofGP = 0
        grades  = {'A+': 4.3, 'A':4, 'A-':3.7, 'B+':3.3, 'B':3, 'B-':2.7, 'C+':2.3, 'C':2, 'C-':1.7, 'D+':1.3, 'D' : 1, 'D-':0.7, 'F':0}
    
        for row in data:
            #print(row)
            if(row[2] == None): #when grade is null. don't count
                continue
            creds += row[1]
            sumofGP += Decimal(grades[row[2]]) * row[1]
            
        if(creds != 0):
            GPA = round(sumofGP / creds, 6)
        else:
            GPA = -1
            
        print("You are a member of %s" % dept)
        print("You have taken total %d credits" %creds)
        print("\n")
        
        if(creds != 0):
            print("GPA : ", GPA)
        else:
            print("GPA : ", "X")
        
        if(GPA == -1):
            print("You Haven't take any courses")
        elif(GPA < 2.0):
            print("Have a meeting with your advisor.")
        elif(GPA < 3.0):
            print("Get better grades by studying more.")
        elif(GPA < 4.0):
            print("Next semester over 4.0 get a scholarship!")
        else:
            print("Apply for a scholarship!")
            
        
    except Exception as err:
        print(err)
        db.rollback()
    
    
def instructor_menu1(instructor, db):
    #TODO modify below(함수에 아무것도 없으면 오류가 생겨서 아무 문장이나 적어놨습니다.)
    year="111"
    semester="????"
    try:
        
        c = db.cursor()
        c.execute("SELECT * FROM classroom;")
        classroom = c.fetchall()
        
        c.execute("SELECT * FROM time_slot;")
        timetable = c.fetchall()
        
        c.execute("SELECT * FROM teaches;")
        data = c.fetchall()
        
        instructor_classes = []
        season = {"Spring" : 0, "Summer" : 1, "Fall":2, "Winter":3}
        recent_season = (0,0)
        
        #check instructor's class and recent_season
        for row in data:
            if(row[0] == instructor.id):
                instructor_classes.append(row)
                if(recent_season == (0,0)): #find first
                    recent_season = (row[3], row[4]) # (0,1)season, year
                else: #compare
                    if(recent_season[1] < row[4]): #year is more earlier
                        recent_season = (row[3], row[4])
                    elif(recent_season[1] == row[4] and season[recent_season[0]] < season[row[3]]): #year is same, more fast season
                        recent_season = (row[3], row[4])
        
        print("Course report - %s %s" % (recent_season[1], recent_season[0]))
        for row in instructor_classes:
            #print(row)
            if(row[3] == recent_season[0] and row[4] == recent_season[1]): #recent semester's course
                #print("this is runnning 2")
                
                c.execute('SELECT * from section natural join course where course_id = "' + row[1] +'" and semester = "'
                         + recent_season[0] + '" and year = "' + str(recent_season[1]) + '" and sec_id = ' + str(row[2]) +';')
                data = c.fetchall()
                
                for row1 in data:
                    print(row1[0], row1[7], "[", row1[4], row1[5], "-", end=" ") #courseid, title, [ building, room_number -
                    for row2 in classroom: # capacity
                        if(row2[0] == row1[4] and row2[1] == row1[5]):
                            print(row2[2], end = " ] ")
                            break
                    temptimetable = [0, 3]
                    print("( ", end = "")
                    for row2 in timetable: #( days time : min - time : min )
                        if(row2[0] == row1[6]):
                            if(temptimetable[0] == row1[6]):
                                print(",", end = "")
                            print(row2[1], end = "")
                            temptimetable = row2
                    print(" " , temptimetable[2] , ":", temptimetable[3] , "-" 
                          , temptimetable[4] , ":" , temptimetable[5] , ")")
                    
                    strFormat = '%-10s%-10s%-15s%-10s'
                    print(strFormat % ('id', 'name', 'dept_name', 'grade'))
                    c.execute('SELECT ID, name, dept_name, grade from student natural join takes where course_id = "' + row[1] +'" and semester = "'
                         + recent_season[0] + '" and year = "' + str(recent_season[1]) + '" and sec_id ='+ str(row[2]) +';')
                
                    studentdata = c.fetchall()
                    for students in studentdata:
                        print(strFormat % students)

        
    except Exception as err:
        print(err)
        db.rollback()
        
        
    

importing Jupyter notebook from user.ipynb
